In [ ]:
import time
from pathlib import Path
import pandas as pd
import numpy as np
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd /content/gdrive/My Drive/버즈니

/content/gdrive/My Drive/버즈니


In [ ]:
train_dataset = pd.read_csv('./recommendation_dataset/user_train.csv')

In [ ]:
train_dataset

,Unnamed: 0,user_id,product_id,data
0,0,1531461682-93ee1a59-e658-4ef5-b4a9-fc8332471e38,964b882e9974101838e0f4907385754c7a76c67804c1b7...,20180714000144
1,0,1531461682-93ee1a59-e658-4ef5-b4a9-fc8332471e38,d4cf98cf780d973a69060b5e75c7e603a36f4d4ec681c1...,20180714000149
2,0,1531461682-93ee1a59-e658-4ef5-b4a9-fc8332471e38,7fddb5bb71072cc2b96ff5cc2aa7a16d961ff1267c0873...,20180714000240
3,0,1531461682-93ee1a59-e658-4ef5-b4a9-fc8332471e38,2586a3ed1bffb041ee8094fe3b126ed5d5c722928cb22c...,20180714000305
4,0,1531461682-93ee1a59-e658-4ef5-b4a9-fc8332471e38,35ea063d443ec3691c2e77c3a76e0c9006cf5d10d9cd26...,20180714000406
...,...,...,...,...
47314,4999,1531633713-7809e3f0-3ff4-4618-b46b-802c1e696826,8aa0dda8cf278c229a03e3aff1ce01eb3a1f2755b8bb46...,20180715235041
47315,4999,1531633713-7809e3f0-3ff4-4618-b46b-802c1e696826,8aa0dda8cf278c229a03e3aff1ce01eb3a1f2755b8bb46...,20180715235059
47316,4999,1531633713-7809e3f0-3ff4-4618-b46b-802c1e696826,8aa0dda8cf278c229a03e3aff1ce01eb3a1f2755b8bb46...,20180715235125
47317,4999,1531633713-7809e3f0-3ff4-4618-b46b-802c1e696826,8aa0dda8cf278c229a03e3aff1ce01eb3a1f2755b8bb46...,20180715235146


In [ ]:
class SessionDataset:
    def __init__(self, path, sep='\t', session_key='user_id', item_key='product_id', time_key='data', n_samples=-1, itemmap=None, time_sort=False):
        """
        Args:
            path: path of the csv file
            sep: separator for the csv
            session_key, item_key, time_key: name of the fields corresponding to the sessions, items, time
            n_samples: the number of samples to use. If -1, use the whole dataset.
            itemmap: mapping between item IDs and item indices
            time_sort: whether to sort the sessions by time or not
        """
        self.df = pd.read_csv(path, names=[session_key, item_key, time_key],low_memory=False)
        self.session_key = session_key
        self.item_key = item_key
        self.time_key = time_key
        self.time_sort = time_sort
        
        # sampling
        if n_samples > 0: self.df = self.df[:n_samples] 
        # Add item indices
        self.add_item_indices(itemmap=itemmap)
        """
        Sort the df by time, and then by session ID. That is, df is sorted by session ID and
        clicks within a session are next to each other, where the clicks within a session are time-ordered.
        """
        self.df.sort_values([session_key, time_key], inplace=True)
         
        self.click_offsets = self.get_click_offsets()
        self.session_idx_arr = self.order_session_idx()
        
        
    def get_click_offsets(self):
        """
        Return the offsets of the beginning clicks of each session IDs,
        where the offset is calculated against the first click of the first session ID.
        """
        offsets = np.zeros(self.df[self.session_key].nunique() + 1, dtype=np.int32)
        # group & sort the df by session_key and get the offset values
        offsets[1:] = self.df.groupby(self.session_key).size().cumsum()

        return offsets
    

    def order_session_idx(self):
        """ Order the session indices """
        if self.time_sort:
            # starting time for each sessions, sorted by session IDs
            sessions_start_time = self.df.groupby(self.session_key)[self.time_key].min().values
            # order the session indices by session starting times
            session_idx_arr = np.argsort(sessions_start_time)
        else:
            session_idx_arr = np.arange(self.df[self.session_key].nunique())

        return session_idx_arr
    
    
    def add_item_indices(self, itemmap=None):
        """ 
        Add item index column named "item_idx" to the df
        Args:
            itemmap (pd.DataFrame): mapping between the item Ids and indices
        """
        if itemmap is None:
            item_ids = self.df[self.item_key].unique()  # unique item ids
            item2idx = pd.Series(data=np.arange(len(item_ids)),
                                 index=item_ids)
            itemmap = pd.DataFrame({self.item_key:item_ids,
                                   'item_idx':item2idx[item_ids].values})
        
        self.itemmap = itemmap
        self.df = pd.merge(self.df, self.itemmap, on=self.item_key, how='inner')
        
    
    @property    
    def items(self):
        return self.itemmap.product_id.unique()

In [ ]:
class SessionDataLoader:
    def __init__(self, dataset, batch_size=50):
        """
        A class for creating session-parallel mini-batches.
        Args:
             dataset (SessionDataset): the session dataset to generate the batches from
             batch_size (int): size of the batch
        """
        self.dataset = dataset
        self.batch_size = batch_size
        
        
    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,): torch.FloatTensor. Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        # initializations
        df = self.dataset.df
        click_offsets = self.dataset.click_offsets
        session_idx_arr = self.dataset.session_idx_arr

        iters = np.arange(self.batch_size)
        maxiter = iters.max()
        start = click_offsets[session_idx_arr[iters]]
        end = click_offsets[session_idx_arr[iters] + 1]
        mask = [] # indicator for the sessions to be terminated
        finished = False
        
        item_idx = df.item_idx.values

        while not finished:
            minlen = (end - start).min()
            # Item indices(for embedding) for clicks where the first sessions start
            idx_target = item_idx[start]
            for i in range(minlen - 1):
                # Build inputs & targets
                idx_input = idx_target
                idx_target = item_idx[start + i + 1]
                input = torch.LongTensor(idx_input)
                target = torch.LongTensor(idx_target)
                # stop flushing the hidden state after the first step
                if i == 1: mask = []
                yield input, target, mask
                
            # click indices where a particular session meets second-to-last element
            start = start + (minlen - 1)
            # see if how many sessions should terminate
            mask = np.arange(len(iters))[(end - start) <= 1]
            for idx in mask:
                maxiter += 1
                if maxiter >= len(click_offsets) - 1:
                    finished = True
                    break
                # update the next starting/ending point
                iters[idx] = maxiter
                start[idx] = click_offsets[session_idx_arr[maxiter]]
                end[idx] = click_offsets[session_idx_arr[maxiter] + 1]

In [ ]:
class Optimizer:
    def __init__(self, params, optimizer_type='Adagrad', lr=.05,
                 momentum=0, weight_decay=0, eps=1e-6):
        '''
        An abstract optimizer class for handling various kinds of optimizers.
        You can specify the optimizer type and related parameters as you want.
        Usage is exactly the same as an instance of torch.optim
        Args:
            params: torch.nn.Parameter. The NN parameters to optimize
            optimizer_type: type of the optimizer to use
            lr: learning rate
            momentum: momentum, if needed
            weight_decay: weight decay, if needed. Equivalent to L2 regulariztion.
            eps: eps parameter, if needed.
        '''
        if optimizer_type == 'RMSProp':
            self.optimizer = optim.RMSprop(params, lr=lr,
                                           eps=eps,
                                           weight_decay=weight_decay,
                                           momentum=momentum)
        elif optimizer_type == 'Adagrad':
            self.optimizer = optim.Adagrad(params, lr=lr,
                                           weight_decay=weight_decay)
        elif optimizer_type == 'Adadelta':
            self.optimizer = optim.Adadelta(params,
                                            lr=lr,
                                            eps=eps,
                                            weight_decay=weight_decay)
        elif optimizer_type == 'Adam':
            self.optimizer = optim.Adam(params,
                                        lr=lr,
                                        eps=eps,
                                        weight_decay=weight_decay)

        elif optimizer_type == 'SparseAdam':
            self.optimizer = optim.SparseAdam(params,
                                              lr=lr,
                                              eps=eps)
        elif optimizer_type == 'SGD':
            self.optimizer = optim.SGD(params,
                                       lr=lr,
                                       momentum=momentum,
                                       weight_decay=weight_decay)
        else:
            raise NotImplementedError
                 
    def zero_grad(self):
        self.optimizer.zero_grad()
    
    def step(self):
        self.optimizer.step()

In [ ]:
class GRU(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, num_layers=1,
                 p_dropout_hidden=0, p_dropout_input=0, batch_size=50, use_cuda=True):
        '''
        The GRU layer used for the whole GRU4REC model.
        Args:
            input_size (int): input layer dimension
            hidden_size (int): hidden layer dimension
            output_size (int): output layer dimension. Equivalent to the number of classes
            num_layers (int): the number of GRU layers
            p_dropout_hidden (float): dropout probability for the GRU hidden layers
            p_dropout_input (float): dropout probability for the GRU input layer
            batch_size (int): size of the training batch.(required for producing one-hot encodings efficiently)
            use_cuda (bool): whether to use cuda or not
            training (bool): whether to set the GRU module to training mode or not. If false, parameters will not be updated.
        '''

        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.p_dropout_input = p_dropout_input
        self.dropout_hidden = nn.Dropout(p_dropout_hidden)

        self.batch_size = batch_size
        self.use_cuda = use_cuda
        self.device = torch.device('cuda' if use_cuda else 'cpu')

        self.onehot_buffer = self.init_emb()  # the buffer where the one-hot encodings will be produced from
        self.h2o = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, dropout=p_dropout_hidden if num_layers > 1 else 0)
        
        self = self.to(self.device)
        

    def forward(self, input, target, hidden):
        '''
        Args:
            input (B,): a batch of item indices from a session-parallel mini-batch.
            target (B,): torch.LongTensor of next item indices from a session-parallel mini-batch.
            
        Returns:
            logit (B,C): Variable that stores the logits for the next items in the session-parallel mini-batch
            hidden: GRU hidden state
        '''
        embedded = self.onehot_encode(input)
        if self.training and self.p_dropout_input > 0: embedded = self.input_dropout(embedded)
        embedded = embedded.unsqueeze(0)  # (1,B,C)

        # Go through the GRU layer
        output, hidden = self.gru(embedded, hidden)  # (num_layers,B,H)
        output = output.view(-1, output.size(-1))  # (B,H)
        output = self.dropout_hidden(output) # hidden layer dropout
        logit = self.tanh(self.h2o(output))  # (B,C)

        return logit, hidden
    
    
    def input_dropout(self, input):
        p_drop = torch.Tensor(input.size(0), 1).fill_(1 - self.p_dropout_input)  # (B,1)
        mask = torch.bernoulli(p_drop).expand_as(input)/(1-self.p_dropout_input) # (B,C)
        mask = mask.to(self.device)
        input = input * mask  # (B,C)
        
        return input
        

    def init_emb(self):
        '''
        Initialize the one_hot embedding buffer, which will be used for producing the one-hot embeddings efficiently
        '''
        onehot_buffer = torch.FloatTensor(self.batch_size, self.output_size)
        onehot_buffer = onehot_buffer.to(self.device)

        return onehot_buffer
    
    
    def onehot_encode(self, input):
        """
        Returns a one-hot vector corresponding to the input
        Args:
            input (B,): torch.LongTensor of item indices
            buffer (B,output_size): buffer that stores the one-hot vector
        Returns:
            one_hot (B,C): torch.FloatTensor of one-hot vectors
        """
        
        self.onehot_buffer.zero_()
        index = input.view(-1,1)
        one_hot = self.onehot_buffer.scatter_(1, index, 1)
        
        return one_hot

    
    def init_hidden(self):
        '''
        Initialize the hidden state of the GRU
        '''
        h0 = torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(self.device)

        return h0

In [ ]:
def LossFunction(loss_type):
    if loss_type == 'CrossEntropy':
        loss_fn = SampledCrossEntropyLoss
    elif loss_type == 'TOP1':
        loss_fn = TOP1Loss
    elif loss_type == 'BPR':
        loss_fn = BPRLoss
    elif loss_type == 'TOP1-max':
        loss_fn = TOP1_max
    elif loss_type == 'BPR-max':
        loss_fn = BPR_max
    else:
        raise NotImplementedError
    return loss_fn


xe_loss = nn.CrossEntropyLoss()
def SampledCrossEntropyLoss(logit):
    """ CrossEntropyLoss with n_classes = batch_size = the number of samples in the session-parallel mini-batch """
    batch_size = logit.size(1)
    target = torch.arange(batch_size).long().to(logit.device)
    return xe_loss(logit, target)


def BPRLoss(logit):
    """
    Args:
        logit (BxB): Variable that stores the logits for the items in the session-parallel mini-batch.
                     Negative samples for a specific item are drawn from the other items in the
                     session-parallel minibatch, as mentioned in the original GRU4REC paper.
                     The first dimension corresponds to the batches, and the second dimension
                     corresponds to sampled number of items to evaluate.
    """
    # differences between the item scores
    diff = logit.diag().view(-1, 1).expand_as(logit) - logit
    # final loss
    loss = -torch.mean(F.logsigmoid(diff))

    return loss

def BPR_max(logit) :
    logit_softmax = F.softmax(logit, dim=1)
    diff = logit.diag().view(-1, 1).expand_as(logit) - logit
    loss = -torch.log(torch.mean(logit_softmax * torch.sigmoid(diff)))
    return loss

    

    
def TOP1Loss(logit):
    """
    Args:
        logit (BxB): Variable that stores the logits for the items in the session-parallel mini-batch.
                     Negative samples for a specific item are drawn from the other items in the
                     session-parallel minibatch, as mentioned in the original GRU4REC paper.
                     The first dimension corresponds to the batches, and the second dimension
                     corresponds to sampled number of items to evaluate.
    """
    # differences between the item scores
    diff = -(logit.diag().view(-1, 1).expand_as(logit) - logit)
    # final loss
    loss = F.sigmoid(diff).mean() + F.sigmoid(logit ** 2).mean()

    return loss

def TOP1_max(logit) :

    logit_softmax = F.softmax(logit, dim=1)
    diff = -(logit.diag().view(-1, 1).expand_as(logit) - logit)
    loss = torch.mean(logit_softmax * (torch.sigmoid(diff) + torch.sigmoid(logit ** 2)))
    return loss

In [ ]:
def get_recall(indices, targets):
    """ Calculates the recall score for the given predictions and targets
    Args:
        indices (Bxk): torch.LongTensor. top-k indices predicted by the model.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        recall (float): the recall score
    """
    targets = targets.view(-1, 1).expand_as(indices)  # (Bxk)
    hits = (targets == indices).nonzero()
    if len(hits) == 0: return 0
    n_hits = (targets == indices).nonzero()[:, :-1].size(0)
    recall = n_hits / targets.size(0)
    
    return recall

def get_precision(indices, targets) :
    """ Calculates the precision score for the given predictions and targets
        indices (Bxk): torch.LongTensor. top-k indices predicted by the model.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        precision (float): the precision score
    """    
    targets = targets.view(-1, 1).expand_as(indices)  # (Bxk)
    hits = (targets == indices).nonzero()
    if len(hits) == 0: return 0
    n_hits = (targets == indices).nonzero()[:, :-1].size(0)
    precision = n_hits / indices.size(-1)
    return precision

def get_f1score(recall,precision) :

    f1score = 2 * (precision * recall / (precision + recall))
    return f1score

def get_mcc(indices, targets) :

    targets = targets.view(-1, 1).expand_as(indices)  # (Bxk)
    hits = (targets == indices).nonzero()
    if len(hits) == 0: return 0
    n_hits = (targets == indices).nonzero()[:, :-1].size(0)

    targets.size(0) * indices.size(-1) *
    mcc = 

def get_mrr(indices, targets):
    """ Calculates the MRR score for the given predictions and targets
    
    Args:
        indices (Bxk): torch.LongTensor. top-k indices predicted by the model.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        mrr (float): the mrr score
    """
    targets = targets.view(-1,1).expand_as(indices)
    # ranks of the targets, if it appears in your indices
    hits = (targets == indices).nonzero()
    if len(hits) == 0: return 0
    ranks = hits[:, -1] + 1
    ranks = ranks.float()
    rranks = torch.reciprocal(ranks)  # reciprocal ranks
    mrr = torch.sum(rranks).data / targets.size(0)
    mrr = mrr.item()
    
    return mrr


def evaluate(logits, targets, k=20):
    """ Evaluates the model using Recall@K, MRR@K scores.
    Args:
        logits (B,C): torch.LongTensor. The predicted logit for the next items.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        recall (float): the recall score
        mrr (float): the mrr score
    """
    _, indices = torch.topk(logits, k, -1)
    recall = get_recall(indices, targets)
    precision = get_precision(indices,targets)
    mrr = get_mrr(indices, targets)

    return recall, precision, mrr

In [ ]:
class GRU4REC:
    def __init__(self, input_size, hidden_size, output_size, num_layers=1,
                 optimizer_type='Adagrad', lr=.01, weight_decay=0,
                 momentum=0, eps=1e-6, loss_type='TOP1',
                 clip_grad=-1, p_dropout_input=.0, p_dropout_hidden=.5,
                 batch_size=50, use_cuda=True, time_sort=False, pretrained=None):
        """ The GRU4REC model
        Args:
            input_size (int): dimension of the gru input variables
            hidden_size (int): dimension of the gru hidden units
            output_size (int): dimension of the gru output variables
            num_layers (int): the number of layers in the GRU
            optimizer_type (str): optimizer type for GRU weights
            lr (float): learning rate for the optimizer
            weight_decay (float): weight decay for the optimizer
            momentum (float): momentum for the optimizer
            eps (float): eps for the optimizer
            loss_type (str): type of the loss function to use
            clip_grad (float): clip the gradient norm at clip_grad. No clipping if clip_grad = -1
            p_dropout_input (float): dropout probability for the input layer
            p_dropout_hidden (float): dropout probability for the hidden layer
            batch_size (int): mini-batch size
            use_cuda (bool): whether you want to use cuda or not
            time_sort (bool): whether to ensure the the order of sessions is chronological (default: False)
            pretrained (modules.layer.GRU): pretrained GRU layer, if it exists (default: None)
        """
        
        # Initialize the GRU Layer
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.use_cuda = use_cuda
        self.device = torch.device('cuda' if use_cuda else 'cpu')
        if pretrained is None:
            self.gru = GRU(input_size, hidden_size, output_size, num_layers,
                           p_dropout_input=p_dropout_input,
                           p_dropout_hidden=p_dropout_hidden,
                           batch_size=batch_size,
                           use_cuda=use_cuda)
        else:
            self.gru = pretrained

        # Initialize the optimizer
        self.optimizer_type = optimizer_type
        self.weight_decay = weight_decay
        self.momentum = momentum
        self.lr = lr
        self.eps = eps
        self.optimizer = Optimizer(self.gru.parameters(),
                                   optimizer_type=optimizer_type,
                                   lr=lr,
                                   weight_decay=weight_decay,
                                   momentum=momentum,
                                   eps=eps)

        # Initialize the loss function
        self.loss_type = loss_type
        self.loss_fn = LossFunction(loss_type)

        # gradient clipping(optional)
        self.clip_grad = clip_grad 

        # etc
        self.time_sort = time_sort
        
        
    def run_epoch(self, dataset, k=20, training=True):
        """ Run a single training epoch """
        start_time = time.time()
        
        # initialize
        losses = []
        recalls = []
        precisions = []
        mrrs = []
        optimizer = self.optimizer
        hidden = self.gru.init_hidden()
        if not training:
            self.gru.eval()
        device = self.device
        
        def reset_hidden(hidden, mask):
            """Helper function that resets hidden state when some sessions terminate"""
            if len(mask) != 0:
                hidden[:, mask, :] = 0
            
            return hidden

        # Start the training loop
        loader = SessionDataLoader(dataset, batch_size=self.batch_size)

        for input, target, mask in loader:
            input = input.to(device)
            target = target.to(device)
            # reset the hidden states if some sessions have just terminated
            hidden = reset_hidden(hidden, mask).detach()
            # Go through the GRU layer
            logit, hidden = self.gru(input, target, hidden)
            # Output sampling
            logit_sampled = logit[:, target.view(-1)]
            # Calculate the mini-batch loss
            loss = self.loss_fn(logit_sampled)
            with torch.no_grad():
                recall, precision,mrr = evaluate(logit, target, k)
            losses.append(loss.item())         
            recalls.append(recall)
            precisions.append(precision)
            mrrs.append(mrr)
            # Gradient Clipping(Optional)
            if self.clip_grad != -1:
                for p in self.gru.parameters():
                    p.grad.data.clamp_(max=self.clip_grad)
            # Mini-batch GD
            if training:
                # Backprop
                loss.backward()
                optimizer.step()
                optimizer.zero_grad() # flush the gradient after the optimization

        results = dict()
        results['loss'] = np.mean(losses)
        results['recall'] = np.mean(recalls)
        results['precision'] = np.mean(precisions)
        results['mrr'] = np.mean(mrrs)
        
        end_time = time.time()
        results['time'] = (end_time - start_time) / 60
        
        if not training:
            self.gru.train()

        return results
    
    
    def train(self, dataset, k=20, n_epochs=10, save_dir='./models', save=True, model_name='GRU4REC'):
        """
        Train the GRU4REC model on a pandas dataframe for several training epochs,
        and store the intermediate models to the user-specified directory.
        Args:
            n_epochs (int): the number of training epochs to run
            save_dir (str): the path to save the intermediate trained models
            model_name (str): name of the model
        """
        print(f'Training {model_name}...')
        for epoch in range(n_epochs):
            results = self.run_epoch(dataset, k=k, training=True)
            results = [f'{k}:{v:.3f}' for k, v in results.items()]
            print(f'epoch:{epoch+1:2d}/{"/".join(results)}')
            
            # Store the intermediate model
            if save:
                save_dir = Path(save_dir)
                if not save_dir.exists(): save_dir.mkdir()
                model_fname = f'{model_name}_{self.loss_type}_{self.optimizer_type}_{self.lr}_epoch{epoch+1:d}'
                torch.save(self.gru.state_dict(), save_dir/model_fname)
    

    def test(self, dataset, k=20):
        """ Model evaluation
        Args:
            k (int): the length of the recommendation list
        Returns:
            avg_loss: mean of the losses over the session-parallel minibatches
            avg_recall: mean of the Recall@K over the session-parallel mini-batches
            avg_precision: mean of the Precision@K over the session-parallel mini-batches
            avg_mrr: mean of the MRR@K over the session-parallel mini-batches
            wall_clock: time took for testing
        """ 
        results = self.run_epoch(dataset, k=k, training=False)
        results = [f'{k}:{v:.3f}' for k, v in results.items()]
        print(f'Test result: {"/".join(results)}')

In [ ]:
train_dataset = SessionDataset('./recommendation_dataset/user_train.csv')

In [ ]:
train_dataset

In [ ]:
input_size = len(train_dataset.items)
hidden_size = 100
num_layers = 1
output_size = input_size
batch_size = 50

optimizer_type = 'Adagrad'
lr = .01
weight_decay = 0
momentum = 0
eps = 1e-6

loss_type = 'TOP1-max'

n_epochs = 10
use_cuda = True

torch.manual_seed(7)

model = GRU4REC(input_size, hidden_size, output_size,
                num_layers=num_layers,
                batch_size=batch_size,
                optimizer_type=optimizer_type,
                lr=lr,
                weight_decay=weight_decay,
                momentum=momentum,
                eps=eps,
                loss_type=loss_type,
                use_cuda=use_cuda)

model_name = 'GRU4REC'
model.train(train_dataset, n_epochs=n_epochs, model_name=model_name, save=False)

Training GRU4REC...
epoch: 1/loss:0.020/recall:0.115/precision:0.289/mrr:0.089/time:0.040
epoch: 2/loss:0.019/recall:0.299/precision:0.748/mrr:0.227/time:0.041
epoch: 3/loss:0.019/recall:0.396/precision:0.990/mrr:0.301/time:0.041
epoch: 4/loss:0.018/recall:0.457/precision:1.141/mrr:0.344/time:0.040
epoch: 5/loss:0.018/recall:0.508/precision:1.269/mrr:0.374/time:0.040
epoch: 6/loss:0.018/recall:0.550/precision:1.376/mrr:0.399/time:0.040
epoch: 7/loss:0.018/recall:0.591/precision:1.478/mrr:0.422/time:0.040
epoch: 8/loss:0.018/recall:0.626/precision:1.564/mrr:0.442/time:0.040
epoch: 9/loss:0.018/recall:0.657/precision:1.643/mrr:0.459/time:0.040
epoch:10/loss:0.018/recall:0.682/precision:1.704/mrr:0.475/time:0.040


In [ ]:
test_dataset = SessionDataset('./recommendation_dataset/user_test.csv', itemmap=train_dataset.itemmap)

In [ ]:
k = 20
model.test(test_dataset, k=k)

Test result: loss:0.019/recall:0.247/precision:0.617/mrr:0.169/time:0.007


In [ ]:
# input_size = len(train_dataset.items)
# hidden_size = 100
# num_layers = 1
# output_size = input_size
# batch_size = 50

# optimizer_type = 'Adagrad'
# lr = .01
# weight_decay = 0
# momentum = 0
# eps = 1e-6

# loss_type = 'BPR'

# n_epochs = 10
# use_cuda = True

# torch.manual_seed(7)

# model = GRU4REC(input_size, hidden_size, output_size,
#                 num_layers=num_layers,
#                 batch_size=batch_size,
#                 optimizer_type=optimizer_type,
#                 lr=lr,
#                 weight_decay=weight_decay,
#                 momentum=momentum,
#                 eps=eps,
#                 loss_type=loss_type,
#                 use_cuda=use_cuda)

# model_name = 'GRU4REC'
# model.train(train_dataset, n_epochs=n_epochs, model_name=model_name, save=False)

In [ ]:
k = 20
model.test(test_dataset, k=k)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Test result: loss:1.002/recall:0.076/mrr:0.042/time:0.026
